# 02 — Feature Engineering & Modeling
Lag/lead engineering, rolling stats, calendar/holiday features, store meta, then train:

- SARIMAX per series (optionally parallel)

- Prophet with regressors

- Global GBM (XGBoost/LightGBM) with time‑aware CV


In [2]:

import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
RAW = Path("../data/raw")

train = pd.read_csv(RAW/"train.csv")
stores = pd.read_csv(RAW/"stores.csv")
features = pd.read_csv(RAW/"features.csv")
for df in [train, features]:
    df['Date'] = pd.to_datetime(df['Date'])

df = (train.merge(stores, on='Store', how='left')
           .merge(features, on=['Store','Date'], how='left')
      ).sort_values(['Store','Dept','Date'])
df = df.drop(columns=['IsHoliday_y'])  # remove duplicate IsHoliday
df = df.rename(columns={'IsHoliday_x': 'IsHoliday'})
df['IsHoliday'] = df['IsHoliday'].astype(bool)

# Calendar features
df['Year'] = df['Date'].dt.year
df['Week'] = df['Date'].dt.isocalendar().week.astype(int)
df['Month'] = df['Date'].dt.month
df['Quarter'] = df['Date'].dt.quarter

# Lags & rolling stats
for k in [1,2,3,4,8,13,26,52]:
    df[f'lag_{k}'] = df.groupby(['Store','Dept'])['Weekly_Sales'].shift(k)
    df[f'roll_mean_{k}'] = df.groupby(['Store','Dept'])['Weekly_Sales'].transform(lambda s: s.shift(1).rolling(k).mean())
    df[f'roll_std_{k}'] = df.groupby(['Store','Dept'])['Weekly_Sales'].transform(lambda s: s.shift(1).rolling(k).std())

# Markdown lag/leads
md_cols = [c for c in df.columns if c.startswith("MarkDown")]
for c in md_cols:
    for k in [1,2,3,4]:
        df[f'{c}_lag{k}'] = df.groupby(['Store','Dept'])[c].shift(k)
        df[f'{c}_lead{k}'] = df.groupby(['Store','Dept'])[c].shift(-k)

# Train/valid split by time
cutoff = df['Date'].quantile(0.8)
train_df = df[df['Date'] <= cutoff].copy()
valid_df = df[df['Date'] > cutoff].copy()

# Feature list (example)
y = 'Weekly_Sales'
drop_cols = ['Date','Weekly_Sales']
X_cols = [c for c in df.columns if c not in drop_cols]
X_cols = [c for c in X_cols if df[c].dtype != 'O']  # numeric only for GBM example
len(X_cols), X_cols[:10]


(81,
 ['Store',
  'Dept',
  'IsHoliday',
  'Size',
  'Temperature',
  'Fuel_Price',
  'MarkDown1',
  'MarkDown2',
  'MarkDown3',
  'MarkDown4'])

In [ ]:
# Global model (LightGBM) with time‑aware training
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.base import clone

def wape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / (np.abs(y_true).sum() + 1e-9)

gbm = LGBMRegressor(n_estimators=1500, learning_rate=0.02, num_leaves=64, subsample=0.8, colsample_bytree=0.8, random_state=42)
gbm.fit(train_df[X_cols], train_df[y])
pred = gbm.predict(valid_df[X_cols])
print("Holdout WAPE:", wape(valid_df[y].values, pred))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.077571 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 18574
[LightGBM] [Info] Number of data points in the train set: 338738, number of used features: 80
[LightGBM] [Info] Start training from score 16029.159427
Holdout WAPE: 0.08190397398508038


In [12]:

# SARIMAX template for a single series (extend with joblib for parallel)
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.statespace.sarimax import SARIMAX

# example = df[(df['Store']==1) & (df['Dept']==1)].copy()
# s = example.set_index('Date')['Weekly_Sales'].asfreq('W-MON')
# exo_cols = ['IsHoliday']
# exo = example.set_index('Date')[exo_cols].asfreq('W-MON')
# exo = example.set_index('Date')[exo_cols].asfreq('W-MON').astype(int)
# exo = exo.reindex(s.index).fillna(0)

# Subset once, set the index once
ex = (df.loc[(df['Store'] == 1) & (df['Dept'] == 1)]
        .set_index('Date')
        .sort_index())

# Weekly target (Mondays)
s = ex['Weekly_Sales'].asfreq('W-MON')

# Exogenous features aligned to s, integers
exo = (ex.loc[:, ['IsHoliday']]
         .asfreq('W-FRI')
         .reindex(s.index)
         .fillna(0)
         .astype(int))

mod = SARIMAX(s, order=(1,1,1), seasonal_order=(1,1,1,52), exog=exo).fit(disp=False)
print(mod.summary())


                                     SARIMAX Results                                      
Dep. Variable:                       Weekly_Sales   No. Observations:                  142
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 52)   Log Likelihood                   0.000
Date:                            Wed, 24 Sep 2025   AIC                             12.000
Time:                                    23:35:41   BIC                             26.932
Sample:                                02-08-2010   HQIC                            18.019
                                     - 10-22-2012                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
IsHoliday           0         -0        nan        nan           0           0
ar.L1               0         -0   

In [13]:

# Prophet with regressors (per series)
from prophet import Prophet

ex = example.copy()
ex = ex.set_index('Date').asfreq('W-FRI')
ex = ex[['Weekly_Sales','IsHoliday']].reset_index().rename(columns={'Date':'ds','Weekly_Sales':'y'})
m = Prophet(weekly_seasonality=False, yearly_seasonality=True, changepoint_prior_scale=0.5)
m.add_regressor('IsHoliday')
m.fit(ex[['ds','y','IsHoliday']])
future = m.make_future_dataframe(periods=13, freq='W-FRI', include_history=False)
future['IsHoliday'] = 0  # replace with planned calendar
fcst = m.predict(future)
fcst[['ds','yhat']].head()


23:36:46 - cmdstanpy - INFO - Chain [1] start processing
23:36:46 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat
0,2012-11-02,32815.511242
1,2012-11-09,27227.340643
2,2012-11-16,19321.223530
3,2012-11-23,16403.106685
4,2012-11-30,22396.628370
